In [1]:
# DEEP BELEIF NETWORK (DBN)
import torch
import torch.nn as nn
import torch.optim as optim
class DBN(nn.Module):
    def __init__(self, layer_sizes):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(layer_sizes[i], layer_sizes[i+1]) for i in range(len(layer_sizes) - 1)])
    def forward(self, x):
        for layer in self.layers:
            x = torch.sigmoid(layer(x))
        return x
    def fine_tune(self, X, y, epochs=5, lr=0.01):
        self.fc = nn.Linear(self.layers[-1].out_features, len(torch.unique(torch.Tensor(y))))
        opt, loss_fn = optim.Adam(self.parameters(), lr=lr), nn.CrossEntropyLoss()
        data, targets = torch.Tensor(X), torch.LongTensor(y)
        for epoch in range(epochs):
            opt.zero_grad()
            loss = loss_fn(self.fc(self(data)), targets)
            loss.backward()
            opt.step()
        print("Weights after training (final layer):")
        print(self.fc.weight)
layer_sizes, X_train, y_train = [784, 512, 256, 128], torch.randn(100, 784), torch.randint(0, 10, (100,))
dbn = DBN(layer_sizes)
dbn.fine_tune(X_train, y_train, epochs=5, lr=0.01)







Weights after training (final layer):
Parameter containing:
tensor([[ 0.0409, -0.0984,  0.0220,  ..., -0.0331, -0.0493,  0.0584],
        [ 0.0257, -0.0430,  0.0198,  ...,  0.0149,  0.1044,  0.0388],
        [ 0.0055,  0.0468, -0.0536,  ...,  0.0462, -0.0384,  0.0566],
        ...,
        [ 0.0192,  0.0657,  0.0078,  ..., -0.0243, -0.0396,  0.0713],
        [ 0.0946,  0.0946,  0.0295,  ...,  0.0060, -0.0691, -0.0602],
        [-0.0607,  0.0398,  0.0181,  ...,  0.0615, -0.0879,  0.0245]],
       requires_grad=True)


In [2]:
# DEEP BOLTZMAN MACHINE (DBM)
import torch
import torch.nn as nn
class DBM(nn.Module):
    def __init__(self, vis_dim=784, hid_dims=[512, 256]):
        super().__init__()
        self.W1 = nn.Parameter(torch.randn(vis_dim, hid_dims[0]) * 0.1)
        self.W2 = nn.Parameter(torch.randn(hid_dims[0], hid_dims[1]) * 0.1)
    def sample(self, prob):
        return torch.bernoulli(prob)
    def forward(self, v):
        h1_p = torch.sigmoid(v @ self.W1)
        return h1_p
if __name__ == "__main__":
    vis_dim, lr, epochs = 784, 0.01, 5
    dbm = DBM(vis_dim)
    opt = torch.optim.Adam(dbm.parameters(), lr=lr)
    for epoch in range(epochs):
        data = torch.randint(0, 2, (32, vis_dim)).float()
        h1_p = dbm(data)
        if epoch == epochs - 1:
            print(f"Hidden Probabilities (h1_p) after training:\n", h1_p)

Hidden Probabilities (h1_p) after training:
 tensor([[0.5285, 0.9618, 0.8458,  ..., 0.3180, 0.3659, 0.0327],
        [0.3057, 0.9100, 0.1911,  ..., 0.2430, 0.2026, 0.1777],
        [0.1861, 0.9123, 0.0597,  ..., 0.4813, 0.3495, 0.0109],
        ...,
        [0.0851, 0.8316, 0.1753,  ..., 0.8759, 0.3257, 0.2980],
        [0.3168, 0.8193, 0.0438,  ..., 0.8137, 0.0257, 0.0286],
        [0.3924, 0.5503, 0.2622,  ..., 0.1562, 0.5962, 0.1955]],
       grad_fn=<SigmoidBackward0>)
